<a href="https://colab.research.google.com/github/ddosaw/oss2025/blob/main/untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#부산 하단동에 있는 모든 편의점 경로 탐색 (동아대 기준)

##1. 부산 사하구 vworld로 지도 불러오기




In [ ]:
import folium

api_key = "9B5EE293-ACF7-311F-98C7-71FF6B101695"


vworld_tile = f"http://api.vworld.kr/req/wmts/1.1.0/{api_key}/Base/{{z}}/{{y}}/{{x}}.png"

# 부산 사하구 중심 좌표
center = [35.1046, 128.9662]

m = folium.Map(location=center, zoom_start=13, tiles=None)

folium.TileLayer(
    tiles=vworld_tile,
    attr='VWorld',
    name='VWorld Base',
    overlay=True,
    control=True
).add_to(m)

m



In [ ]:
!pip install --upgrade osmnx --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 55.5 MB/s eta 0:00:00


In [ ]:
import osmnx as ox
print(ox.__version__)


2.0.2


In [ ]:
import folium
import IPython.display

# 지도 객체 생성

##2.하단동에 있는 모든 편의점 카카오 api로 불러오기

In [ ]:
import requests

kakao_api_key = "d5565a677269c1b8df8c8a125b1874e3"

def search_convenience_stores(keyword="하단동 편의점", size=15):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}
    stores = []
    page = 1

    while True:
        params = {
            "query": keyword,
            "page": page,
            "size": size
        }
        res = requests.get(url, headers=headers, params=params)
        data = res.json()

        if "documents" not in data or len(data["documents"]) == 0:
            break

        for doc in data["documents"]:
            stores.append({
                "name": doc["place_name"],
                "lat": float(doc["y"]),
                "lon": float(doc["x"]),
                "address": doc["road_address_name"]
            })

        if data["meta"]["is_end"]:
            break
        page += 1

    return stores

# 하단동 편의점 정보 가져오기
stores = search_convenience_stores("하단동 편의점")

formatted_stores = [
    { "name": store["name"], "lat": store["lat"], "lon": store["lon"] }
    for store in stores
]

# 출력된 편의점 리스트 보기
for store in formatted_stores:
    print(store)


{'name': 'CU 하단골드점', 'lat': 35.1068210357875, 'lon': 128.965545989746}
{'name': 'GS25 하단프레시점', 'lat': 35.1059055546913, 'lon': 128.961589303464}
{'name': 'GS25 하단센터점', 'lat': 35.1076242690821, 'lon': 128.966473307902}
{'name': 'GS25 하신번영로점', 'lat': 35.1021986240378, 'lon': 128.958044047347}
{'name': 'GS25 부산을숙도점', 'lat': 35.11092011949645, 'lon': 128.94692603615206}
{'name': 'CU 하단블루점', 'lat': 35.11233182694128, 'lon': 128.96176199860508}
{'name': 'GS25 하단낙동점', 'lat': 35.1070271688002, 'lon': 128.963559438217}
{'name': 'GS25 하단사하점', 'lat': 35.1054485374189, 'lon': 128.966600874725}
{'name': 'CU 하단스마일점', 'lat': 35.1064987389956, 'lon': 128.967191974285}
{'name': 'GS25 하단번영점', 'lat': 35.1108540501611, 'lon': 128.966033284969}
{'name': 'GS25 부산가락타운점', 'lat': 35.1038141404109, 'lon': 128.959345978363}
{'name': '이마트24 R하단센터점', 'lat': 35.1071146069163, 'lon': 128.96678128087}
{'name': 'GS25 하단종로점', 'lat': 35.1047800263211, 'lon': 128.965812796582}
{'name': 'GS25 에덴점', 'lat': 35.1124087507277

## 3. T map으로 동아대에서 편의점 모든 경로 설정하기

In [ ]:
import requests


TMAP_API_KEY = "Wz4axgTD4I5hmkozEKgjk1IsI6jutZou9tnL8ZjM"

# 동아대 위치
dong_a_univ = {"lat": 35.1191, "lon": 128.9686}

# TMAP 경로 계산
def get_route(start, end):
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian?version=1"
    headers = {
        "appKey": TMAP_API_KEY,
        "Content-Type": "application/json"
    }
    body = {
        "startX": str(start["lon"]),
        "startY": str(start["lat"]),
        "endX": str(end["lon"]),
        "endY": str(end["lat"]),
        "reqCoordType": "WGS84GEO",
        "resCoordType": "WGS84GEO",
        "startName": "출발지",
        "endName": "도착지"
    }


    res = requests.post(url, headers=headers, json=body)


    try:
        data = res.json()
        features = data.get("features", [])
        route = []
        for feat in features:
            coords = feat["geometry"]["coordinates"]
            if feat["geometry"]["type"] == "LineString":
                for coord in coords:
                    route.append((coord[1], coord[0]))  # (위도, 경도)
        return route
    except Exception as e:
        print(f"경로 계산 오류: {e}")
        return []
stores = search_convenience_stores("하단동 편의점")
# 경로 계산: 동아대와 각 편의점 사이의 경로
routes = []
for store in stores:
    path = get_route(dong_a_univ, store)
    if path:
        routes.append({
            "name": store["name"],
            "lat": store["lat"],
            "lon": store["lon"],
            "route": path
        })

# 결과 확인
print(f"경로 계산 완료된 편의점 개수: {len(routes)}")
for r in routes[:3]:  # 앞 3개 경로만 확인
    print(f"{r['name']} → 경로 길이: {len(r['route'])} 좌표점")


경로 계산 완료된 편의점 개수: 45
CU 하단골드점 → 경로 길이: 72 좌표점
GS25 하단프레시점 → 경로 길이: 90 좌표점
GS25 하단센터점 → 경로 길이: 68 좌표점


##4.folium을 이용해 2d 지도로 동아대와 모든 편의점 경로 나타내기

In [ ]:
import folium

# 동아대 중심 지도
m = folium.Map(location=[35.1191, 128.9686], zoom_start=14, tiles=None)


vworld_tile = f"http://api.vworld.kr/req/wmts/1.1.0/{api_key}/Base/{{z}}/{{y}}/{{x}}.png"
folium.TileLayer(
    tiles=vworld_tile,
    attr='VWorld',
    name='VWorld Base',
    overlay=True,
    control=True
).add_to(m)


folium.Marker(
    location=[dong_a_univ["lat"], dong_a_univ["lon"]],
    popup="동아대학교",
    icon=folium.Icon(color="red")
).add_to(m)

# 편의점 마커 + 경로 선
for r in routes:
    folium.Marker(
        location=[r["lat"], r["lon"]],
        popup=r["name"],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

    folium.PolyLine(
        locations=r["route"],
        color="green",
        weight=3,
        opacity=0.7,
        tooltip=r["name"]
    ).add_to(m)

# 지도 출력
m


In [ ]:
!pip install pydeck

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.7 MB/s eta 0:00:00


In [ ]:
!pip install mapbox


## 5.mapbox로 3d 지도로 표현하기, 계속되는 오류로 html파일을 통해 표현 (상위 10개의 편의점을 직선 경로로 )

In [ ]:
from google.colab import files
import json

html_code = '''<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <title>동아대에서 하단동 편의점까지의 경로</title>
  <link href="https://api.mapbox.com/mapbox-gl-js/v2.15.0/mapbox-gl.css" rel="stylesheet" />
  <style>
    body {{ margin: 0; padding: 0; }}
    #map {{ position: absolute; top: 0; bottom: 0; width: 100%; }}
  </style>
</head>
<body>
  <div id="map"></div>
  <script src="https://api.mapbox.com/mapbox-gl-js/v2.15.0/mapbox-gl.js"></script>
  <script>
    mapboxgl.accessToken = 'pk.eyJ1IjoiZG9uZ2RvbmdzIiwiYSI6ImNtYWFxNWxwYzBnOGYyaXBxMzAyZmJ4N24ifQ.rA9GHJBHWfio2zet8NgBPg';
    const dong_a_univ = {{ lat: 35.1191, lon: 128.9686 }};

    const stores = {};

    const map = new mapboxgl.Map({{
      container: 'map',
      style: 'mapbox://styles/mapbox/streets-v11',
      center: [dong_a_univ.lon, dong_a_univ.lat],
      zoom: 14,
      pitch: 60,
      bearing: -60
    }});

    map.on('load', () => {{
      new mapboxgl.Marker({{ color: 'red' }})
        .setLngLat([dong_a_univ.lon, dong_a_univ.lat])
        .setPopup(new mapboxgl.Popup().setText('동아대'))
        .addTo(map);

      stores.forEach((store, index) => {{
        new mapboxgl.Marker({{ color: 'blue' }})
          .setLngLat([store.lon, store.lat])
          .setPopup(new mapboxgl.Popup().setText(store.name))
          .addTo(map);

        const route = {{
          'type': 'Feature',
          'geometry': {{
            'type': 'LineString',
            'coordinates': [
              [dong_a_univ.lon, dong_a_univ.lat],
              [store.lon, store.lat]
            ]
          }}
        }};

        map.addLayer({{
          'id': `route-${{index}}`,
          'type': 'line',
          'source': {{
            'type': 'geojson',
            'data': route
          }},
          'paint': {{
            'line-color': '#FF5733',
            'line-width': 3
          }}
        }});
      }});
    }});
  </script>
</body>
</html>
'''

#편의점 데이터 10개만 표현
stores_data = [
  {"name": "CU 하단골드점", "lat": 35.1068210357875, "lon": 128.965545989746},
  {"name": "GS25 하단프레시점", "lat": 35.1059055546913, "lon": 128.961589303464},
  {"name": "GS25 하단센터점", "lat": 35.1076242690821, "lon": 128.966473307902},
  {"name": "GS25 하신번영로점", "lat": 35.1021986240378, "lon": 128.958044047347},
  {"name": "GS25 부산을숙도점", "lat": 35.11092011949645, "lon": 128.94692603615206},
  {"name": "CU 하단블루점", "lat": 35.11233182694128, "lon": 128.96176199860508},
  {"name": "GS25 하단낙동점", "lat": 35.1070271688002, "lon": 128.963559438217},
  {"name": "GS25 하단사하점", "lat": 35.1054485374189, "lon": 128.966600874725},
  {"name": "CU 하단스마일점", "lat": 35.1064987389956, "lon": 128.967191974285},
  {"name": "GS25 하단번영점", "lat": 35.1108540501611, "lon": 128.966033284969}
]


html_final = html_code.format(json.dumps(stores_data, ensure_ascii=False))

# 다운로드
file_name = 'dongA_to_convenience_map.html'
with open(file_name, 'w', encoding='utf-8') as f:
    f.write(html_final)

files.download(file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>